# Neural ODEs
Red Neuronal para simulacion de EDOs de primer ordel del tipo
$$\dfrac{d}{dt}S(t)=P_1(t) + P_2(t)*P_3(S)$$
donde $P_1$, $P_2$ y $P_3$ son polinomios.

In [1]:
# Recarga los modulos
%load_ext autoreload
%autoreload 2

In [2]:
import os

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [25]:
import random

In [3]:
import gpustat
gpustat.print_gpustat()

Lucifer                                2022-11-26 13:30:29  526.98
[0] NVIDIA GeForce RTX 3080 Laptop GPU | 49°C,   0 % |   161 /  8192 MB |


In [4]:
import numpy as mp
import pandas as pd

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data as data_torch

In [7]:
from tqdm import tnrange, tqdm_notebook

In [8]:
from api.utils import *

## Tratamiento de datos

In [9]:
# Obtenemos datos de las simulaciones
file_name = 'datos_ODEs_100.csv'
dir_file = 'C:/Users/Det-Pc/OneDrive/Documentos/GitHub/Proyecto_curso_DL/data'
data = get_ode_data(dir_file + '/' + file_name)
# data.head()

El tamano de los datos de polinomio en S y t es: 333274
El tamano de los datos de polinomio en S es    : 333811
El tamano de los datos de polinomio en t es    : 332915
El numero total de los datos es                : 1000000


,funcion,grade_f_t,grade_f_s,I_c,n_steps,int_0,int_f,sol.t,sol.y
0,polinom_t,6,0,0.704150,100,0,1,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.7041497439187168, 0.7044874522964137, 0.705..."
1,polinom_t,7,0,-0.616239,100,0,1,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[-0.6162386742618866, -0.6163800903218846, -0...."
2,polinom_s,0,7,0.098112,100,0,1,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.09811174156903335, 0.09853803527908014, 0.0..."
3,polinom_st,4,2,0.258020,100,0,1,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.2580197483455098, 0.25801897523637685, 0.25..."
4,polinom_st,9,10,-0.660801,100,0,1,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[-0.6608008149233784, -0.6607992730118504, -0...."


In [10]:
# Extraemos los valores de las simulaciones
df_values = data['sol.y'].values

### DataSets

In [22]:
seq_len = 75
size_predic = len(data['sol.y'].values[0])
len_sim = df_values.shape[0]

In [18]:
# Creamos las sucesiones con los datos estandarizados
X_data, Y_data = get_seqs(data=df_values, seq_len=seq_len)

In [19]:
print(f'EL tamano de los datos las sucesiones_X es: {X_data.shape}')
print(f'EL tamano de los datos las sucesiones_Y es: {Y_data.shape}')

EL tamano de los datos las sucesiones_X es: (999925, 75)
EL tamano de los datos las sucesiones_Y es: (999925, 25)


In [26]:
# porcentaje de datos de entrenamiento
pct_data_train = 0.8

In [27]:
index_data = list(range(X_data.shape[0]))

train_index = random.sample(index_data, int(len(index_data)*pct_data_train))
val_test_index = list(set(index_data)-set(train_index))
val_index = random.sample(val_test_index, int(len(val_test_index)*0.5))
test_index = list(set(val_test_index)-set(val_index))

In [28]:
# Creamos las particiones
data_dic = {'train':X_data[train_index],\
           'val':X_data[val_index],\
           'test':X_data[test_index]}

In [41]:
class EDOs_dataloader(data_torch.Dataset):
    def __init__(self, edos_s):

        self.edos_s = edos_s
        
        
    def __getitem__(self, index):
        edo_i = self.edos_s[index]
        return edo_i
        

    def __len__(self):
        return len(self.edos_s)

In [42]:
partitions = ['train', 'val', 'test']
batch_size = 512
shuffle = True
lr = 0.001
edos_datasets = {x: EDOs_dataloader(data_dic[x]) for x in partitions}
dataloaders = {x: torch.utils.data.DataLoader(edos_datasets[x], batch_size=batch_size,\
                                              shuffle=shuffle) for x in partitions}
dataset_sizes = {x: len(edos_datasets[x]) for x in partitions}
print('Se cargo la data exitosamente!')
print(f'El tamano de la dataset es {dataset_sizes}')

Se cargo la data exitosamente!
El tamano de la dataset es {'train': 799940, 'val': 99992, 'test': 99993}


## Modelo

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Estamos usando: {}'.format(device))

Estamos usando: cuda


In [ ]:
class ODE_NET(nn.Module):
    def __init__(self):
        super(ODE_NET, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1,out_channels=32,kernel_size=6,stride=2,padding=3)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(2, 2)
        self.conv2 = nn.Conv1d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.rnn1 = nn.LSTM()

    def forward(self, x):
        print(x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [ ]:
class AutoEncoder_TS(nn.Module):
    def __init__(self):
        super(AutoEncoder_TS, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=1,out_channels=32,kernel_size=6,stride=2,padding=3)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.conv2 = nn.Conv1d(in_channels=32,out_channels=16,kernel_size=6,stride=2,padding=3)
        self.convTrans = nn.ConvTranspose1d(in_channels=16, out_channels=16,kernel_size=6,stride=2,padding=3)
        self.convTrans2 = nn.ConvTranspose1d(in_channels=16, out_channels=32,kernel_size=6,stride=2,padding=2)
        self.convTrans3 = nn.ConvTranspose1d(in_channels=32, out_channels=1,kernel_size=5,stride=1, padding=2)
    def forward(self, x):
        
        x = self.dropout(self.relu(self.conv1(x)))
        x = self.relu(self.conv2(x))
        x = self.dropout(self.relu(self.convTrans(x)))
        x = self.relu(self.convTrans2(x))
        x = self.convTrans3(x)
        return x